<a href="https://colab.research.google.com/github/EnriqueQM/bmw-pricing-proyecto/blob/main/BMW_Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PASOS INICIALES

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## BWM PRICING


In [3]:
df_bmw = pd.read_csv("/content/drive/MyDrive/bmw_pricing_v3.csv", sep=",")
df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01
1,BMW,M4,13929.0,317.0,NaN,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01
3,BMW,420,128035.0,135.0,NaN,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01
4,BMW,425,97097.0,160.0,NaN,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01


In [4]:
df_bmw.shape

(4843, 18)

In [5]:
df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        3873 non-null   object 
 1   modelo                       4840 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               2420 non-null   object 
 5   tipo_gasolina                4838 non-null   object 
 6   color                        4398 non-null   object 
 7   tipo_coche                   3383 non-null   object 
 8   volante_regulable            4839 non-null   object 
 9   aire_acondicionado           4357 non-null   object 
 10  camara_trasera               4841 non-null   object 
 11  asientos_traseros_plegables  1452 non-null   object 
 12  elevalunas_electrico         4841 non-null   object 
 13  bluetooth         

In [6]:
df_bmw.describe()

,km,potencia,precio
count,4.841000e+03,4842.000000,4837.000000
mean,1.409593e+05,128.981826,15831.920612
std,6.020853e+04,38.994839,9222.630708
min,-6.400000e+01,0.000000,100.000000
25%,1.028840e+05,100.000000,10800.000000
50%,1.410800e+05,120.000000,14200.000000
75%,1.752170e+05,135.000000,18600.000000
max,1.000376e+06,423.000000,178500.000000


## TRATAMIENTO DE DUPLICADOS

In [7]:
df_bmw.duplicated().sum()

0

## FECHAS

In [35]:
df_bmw['fecha_registro'].value_counts()

,count
fecha_registro,
2013-07-01,93
2014-03-01,83
2014-05-01,81
2013-01-01,78
2013-10-01,77
...,...
2007-01-01,1
1990-03-01,1
2006-11-01,1


##TRATAMIENTO DE NULOS

In [8]:
df_bmw.isnull().sum()

,0
marca,970
modelo,3
km,2
potencia,1
fecha_registro,2423
tipo_gasolina,5
color,445
tipo_coche,1460
volante_regulable,4
aire_acondicionado,486


MARCA

In [10]:
df_bmw["marca"].value_counts()

,count
marca,
BMW,3873


In [12]:
## Sabemos que es dataset puramente de BWM así que rellenamos los nulos por BMW

df_bmw['marca'].fillna('BMW', inplace=True)

<ipython-input-12-7ef154e1d259>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_bmw['marca'].fillna('BMW', inplace=True)


In [13]:
df_bmw.isnull().sum()

,0
marca,0
modelo,3
km,2
potencia,1
fecha_registro,2423
tipo_gasolina,5
color,445
tipo_coche,1460
volante_regulable,4
aire_acondicionado,486


MODELO

In [14]:
df_bmw["modelo"].value_counts()

,count
modelo,
320,752
520,633
318,568
X3,438
116,358
...,...
M135,1
225,1
i8,1


In [19]:
df_bmw['modelo'].unique()

array(['118', 'M4', '320', '420', '425', '335', '325', 'Z4', '120', '220',
       '650', '135', '218', '318', '430', 'M3', '330', '225', 'M235',
       '316', '435', '640', '640 Gran Coupé', '435 Gran Coupé', 'i8',
       '630', '430 Gran Coupé', '635', '520', nan, '525', '530', '535',
       '320 Gran Turismo', '518', '116', '328', '523', 'X3', 'M550',
       '528', '325 Gran Turismo', '125', '318 Gran Turismo',
       '420 Gran Coupé', 'i3', '114', '520 Gran Turismo',
       '530 Gran Turismo', 'M135', '418 Gran Coupé', '330 Gran Turismo',
       '335 Gran Turismo', '730', '123', '535 Gran Turismo', '740', '750',
       'ActiveHybrid 5', '735', 'M5', 'X5', 'X1', 'X4', 'X6', 'X6 M',
       'X5 M', 'X5 M50', '218 Active Tourer', '218 Gran Tourer', '216',
       '220 Active Tourer', '216 Gran Tourer', '216 Active Tourer',
       ' Active Tourer', '225 Active Tourer', '214 Gran Tourer'],
      dtype=object)

In [20]:
df_bmw['modelo'].fillna('320', inplace=True)

KM

In [24]:
df_bmw['km'].describe()

,km
count,4.841000e+03
mean,1.409593e+05
std,6.020853e+04
min,-6.400000e+01
25%,1.028840e+05
50%,1.410800e+05
75%,1.752170e+05
max,1.000376e+06


In [29]:
df_bmw['km'].fillna(df_bmw['km'].mean(), inplace=True)

<ipython-input-29-82268f1f1b9e>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_bmw['km'].fillna(df_bmw['km'].mean(), inplace=True)


POTENCIA

In [30]:
df_bmw['potencia'].describe()

,potencia
count,4842.000000
mean,128.981826
std,38.994839
min,0.000000
25%,100.000000
50%,120.000000
75%,135.000000
max,423.000000


In [33]:
df_bmw['potencia'].fillna(df_bmw['potencia'].median(), inplace=True)